In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pyautogui
import logging
import os
import shutil
import time

# =================== CONFIGURAÇÕES ===================
# Diretórios
download_dir = r"C:\Users\admin\Downloads"
destino_folder = r"C:\Users\admin\Desktop\Dash Presidência"
arquivo_nome_sharepoint = "extração sustentaçao.xlsx"  # Nome parcial esperado para o arquivo do SharePoint

# URLs e nomes dos arquivos do Redmine
urls_com_filtro = [
    {
        "url": "https://redmine.prodesp.sp.gov.br/projects/portal-detran-sustentacao/issues?query_id=7701",
        "novo_nome": "Portal Abertas - Presidência.csv",
    },
    {
        "url": "https://redmine.prodesp.sp.gov.br/projects/portal-detran-sustentacao/issues?query_id=7702",
        "novo_nome": "Portal Concluídas - Presidência.csv",
    },
    {
        "url": "https://redmine.prodesp.sp.gov.br/projects/suporte/issues?query_id=7699",
        "novo_nome": "Presidência - Abertas.csv",
    },
    {
        "url": "https://redmine.prodesp.sp.gov.br/projects/suporte/issues?query_id=7700",
        "novo_nome": "Presidência - Concluídas.csv",
    }
]

# Configuração de Logs
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")

# =================== PRIMEIRO PROCESSO - REDMINE ===================
def baixar_redmine_arquivos():
    """Baixa arquivos do Redmine e os renomeia."""
    chrome_options = Options()
    prefs = {"download.default_directory": download_dir}
    chrome_options.add_experimental_option("prefs", prefs)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        # Login no Redmine
        logging.info("Acessando o Redmine...")
        driver.get("https://redmine.prodesp.sp.gov.br/login")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))

        driver.find_element(By.ID, "username").send_keys("28875444897")
        driver.find_element(By.ID, "password").send_keys("@15MN09cp77@")
        driver.find_element(By.ID, "login-submit").click()

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "content")))

        # Baixar os arquivos
        for item in urls_com_filtro:
            logging.info(f"Processando URL: {item['url']}")
            driver.get(item["url"])

            # Clicar no botão de exportar CSV
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "csv")]'))
            ).click()

            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//input[@type="submit" and @value="Exportar"]'))
            ).click()

            time.sleep(5)  # Espera o download concluir

            # Renomear e mover o arquivo
            nome_arquivo_baixado = "issues.csv"
            caminho_antigo = os.path.join(download_dir, nome_arquivo_baixado)
            caminho_novo = os.path.join(destino_folder, item["novo_nome"])

            os.makedirs(destino_folder, exist_ok=True)
            if os.path.exists(caminho_antigo):
                shutil.move(caminho_antigo, caminho_novo)
                logging.info(f"Arquivo '{item['novo_nome']}' movido com sucesso.")
            else:
                logging.warning(f"Arquivo '{nome_arquivo_baixado}' não encontrado.")

    except Exception as e:
        logging.error(f"Erro no processo do Redmine: {e}")
    finally:
        driver.quit()

# =================== SEGUNDO PROCESSO - SHAREPOINT ===================
def acessar_sharepoint_interagir():
    """Acessa o SharePoint, realiza as interações e move o arquivo baixado."""
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "safebrowsing.enabled": True
    })

    driver = webdriver.Chrome(options=chrome_options)

    try:
        logging.info("Acessando a página de login do SharePoint...")
        site_url = "https://governosp.sharepoint.com/teams/DETRAN-gestao.sistemas"
        driver.get(site_url)

        time.sleep(5)  # Ajuste o tempo conforme necessário

        # Interação via PyAutoGUI
        pyautogui.PAUSE = 1.0
        time.sleep(5)

        pyautogui.moveTo(x=951, y=813, duration=1)
        pyautogui.click(x=951, y=813)

        time.sleep(8)
        pyautogui.moveTo(x=993, y=537, duration=1)
        pyautogui.click(x=993, y=537)

        time.sleep(8)
        pyautogui.moveTo(x=1068, y=503, duration=1)
        pyautogui.click(x=1068, y=503)

        time.sleep(8)
        # Outras interações
        pyautogui.write("15072016")
        time.sleep(8)
        
        pyautogui.moveTo(x=231, y=579, duration=1)
        pyautogui.click(x=231, y=579)

        time.sleep(8)
        pyautogui.moveTo(x=525, y=573, duration=1)
        pyautogui.click(x=525, y=573)
    
        time.sleep(8)
        pyautogui.moveTo(x=528, y=573, duration=1)
        pyautogui.click(x=528, y=573)

        time.sleep(8)
        pyautogui.moveTo(x=834, y=626, duration=1)
        pyautogui.click(x=834, y=626)

        time.sleep(8)
        pyautogui.moveTo(x=908, y=670, duration=1)
        pyautogui.click(x=908, y=670)

        # Verificar e mover o arquivo
        logging.info("Verificando e movendo o arquivo do SharePoint...")
        time.sleep(30)  # Tempo para garantir o download

        for arquivo in os.listdir(download_dir):
            if arquivo_nome_sharepoint in arquivo:
                origem = os.path.join(download_dir, arquivo)
                destino = os.path.join(destino_folder, arquivo)
                shutil.move(origem, destino)
                logging.info(f"Arquivo '{arquivo}' movido com sucesso.")
                return

        logging.warning("Arquivo do SharePoint não encontrado.")
    except Exception as e:
        logging.error(f"Erro no processo do SharePoint: {e}")
    finally:
        driver.quit()

# =================== FLUXO PRINCIPAL ===================
if __name__ == "__main__":
    try:
        # Primeiro processo: Baixar e mover arquivos do Redmine
        baixar_redmine_arquivos()

        # Segundo processo: Interagir com SharePoint e mover arquivo
        acessar_sharepoint_interagir()

    except Exception as e:
        logging.error(f"Erro no fluxo principal: {e}")

    finally:
        logging.info("Processo concluído.")


2024-12-17 14:23:27,979 - ====== WebDriver manager ======
2024-12-17 14:23:29,655 - Get LATEST chromedriver version for google-chrome
2024-12-17 14:23:29,828 - Get LATEST chromedriver version for google-chrome
2024-12-17 14:23:29,881 - Driver [C:\Users\admin\.wdm\drivers\chromedriver\win64\131.0.6778.108\chromedriver-win32/chromedriver.exe] found in cache
2024-12-17 14:23:31,547 - Acessando o Redmine...
2024-12-17 14:23:34,027 - Processando URL: https://redmine.prodesp.sp.gov.br/projects/portal-detran-sustentacao/issues?query_id=7701
2024-12-17 14:23:40,733 - Arquivo 'Portal Abertas - Presidência.csv' movido com sucesso.
2024-12-17 14:23:40,735 - Processando URL: https://redmine.prodesp.sp.gov.br/projects/portal-detran-sustentacao/issues?query_id=7702
2024-12-17 14:23:48,906 - Arquivo 'Portal Concluídas - Presidência.csv' movido com sucesso.
2024-12-17 14:23:48,907 - Processando URL: https://redmine.prodesp.sp.gov.br/projects/suporte/issues?query_id=7699
2024-12-17 14:23:58,054 - Arqui